# Лабораторная работа №4



1.   Local search - best-improvement
2.  Iterated local search

# Получаем данные

In [ ]:
!wget https://www.dropbox.com/s/h1zlfcbw66vdhkc/qap_test.zip?dl=0 -O qap_test.zip
!unzip '/content/qap_test.zip'

--2022-06-10 07:39:19--  https://www.dropbox.com/s/h1zlfcbw66vdhkc/qap_test.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6022:18::a27d:4212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/h1zlfcbw66vdhkc/qap_test.zip [following]
--2022-06-10 07:39:19--  https://www.dropbox.com/s/raw/h1zlfcbw66vdhkc/qap_test.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uccd595d015907efc247f05f7d63.dl.dropboxusercontent.com/cd/0/inline/Bm5mLaWW7-ZTWtMX8d8lHy3UWrOlfodEP1n-gelrQvhHazOtms4xplaO7A1Ah13vD9yIvsvIcKbuwk8bcJnBcLkZp4gpwU8Ed7Lf-kVmt6u3-gqFrdRZvPzXEku_t2Fms4y9_XFxG-PESfGbxMACVCyskSW7Ah6xZJQciePEgcQoYg/file# [following]
--2022-06-10 07:39:19--  https://uccd595d015907efc247f05f7d63.dl.dropboxusercontent.com/cd/0/inline/Bm5mLaWW7-ZTWtMX8d8lHy3UWrOlfodEP1n-gelrQvhHazOtms4xplaO7A1A

In [ ]:
filenames = []
for i in range(2, 12, 2):
  txt = f'tai{i}0a'
  filenames.append(txt)

In [ ]:
from os import listdir
from os.path import isfile, join
folder = '/content'
data = []
for filename in filenames:
  info_dict = {}
  with open(join(folder, filename), 'r') as f:
    n = f.readline()
    D_list = []
    F_list = []
    info_dict['n'] = int(n)
    for i in range(int(n)):
      D = f.readline()
      D = D.replace('  ', ' ').replace('\n', '')
      D = D.split(' ')[1:]
      D_int = [int(x) for x in D]
      D_list.append(D_int)
    info_dict['D'] = D_list
    f.readline()
    for i in range(int(n)):
      F = f.readline()
      F = F.replace('  ', ' ').replace('\n', '')
      F = F.split(' ')[1:]
      F_int = [int(x) for x in F]
      F_list.append(F_int)

    info_dict['F'] = F_list
  data.append(info_dict)

# Local search - best-improvement

## Жадная эвристика

In [ ]:
import random

In [ ]:
def gready_search(dictance_table, flow_table):
  pass

In [ ]:
def random_init(n):
  # список, где индексы - номера фабрик, значения по индексу - номера мест
  return random.sample([i for i in range(n)], n)

In [ ]:
for item in data:
  print(len(item['D']), len(item['F']))

20 20
40 40
60 60
80 80
100 100


In [ ]:
init_solution = random_init(data[0]['n'])

In [ ]:
from copy import copy

In [ ]:
def count_first_delta(solution, matrix_d, matrix_f, r, s):
  next_solution = copy(solution)
  next_solution[r], next_solution[s] = next_solution[s], next_solution[r]
  s = 0
  for k in range(len(matrix_d)):
    if k == r or k == s:
      continue
    s += (matrix_f[s][k] - matrix_f[r][k]) \
    * (matrix_d[next_solution[s]][next_solution[k]] - matrix_d[solution[r]][solution[k]])

  return 2 * s

In [ ]:
def count_delta(solution, matrix_d, matrix_f, r, s, u, v, prev_delta):
  next_solution = copy(solution)
  next_solution[u], next_solution[v] = next_solution[v], next_solution[u]
  s_1 = (matrix_f[r][u] - matrix_f[r][v] + matrix_f[s][v] - matrix_f[s][u]) * (matrix_d[next_solution[s]][next_solution[u]] - matrix_d[next_solution[s]][next_solution[v]] + matrix_d[next_solution[r]][next_solution[v]] - matrix_d[next_solution[r]][next_solution[u]])
  s_2 = (matrix_f[u][r] - matrix_f[v][r] + matrix_f[v][s] - matrix_f[u][s]) * (matrix_d[next_solution[u]][next_solution[s]] - matrix_d[next_solution[v]][next_solution[s]] + matrix_d[next_solution[v]][next_solution[r]] - matrix_d[next_solution[u]][next_solution[r]])
  s = s_1 + s_2 + prev_delta
  return s

In [ ]:
def count_system_state(matrix_d, matrix_f, n, solution):
  value = 0
  for i in range(n):
      for j in range(n):
          value += matrix_d[i][j] * matrix_f[solution[i]][solution[j]]
  return value

In [ ]:
def local_search(matrix_d, matrix_f, n, solution, last_delta=None, last_r=None, last_s=None):
  saved_deltas = [[None for i in range(n)] for j in range(n)]
  delta = last_delta
  r, s = last_r, last_s
  i, j = 0, 1
  while i < n:
    while j < n:
      if saved_deltas[i][j] != None or i in [r, s] or j in [r, s]:
        j += 1
        continue

      if delta == None:
        delta = count_first_delta(solution, matrix_d, matrix_f, i, j)
        r = i
        s = j
        solution[i], solution[j] = solution[j], solution[i]
        saved_deltas[i][j] = delta
        j += 1
        continue

      new_delta = count_delta(solution, matrix_d, matrix_f, r, s, i, j, delta)
      if new_delta < delta:
        delta = new_delta
        r = i
        s = j
        solution[i], solution[j] = solution[j], solution[i]
        saved_deltas[i][j] = delta
        i = 0
        j = 1

      j += 1

    i += 1
    j = i + 1

  final_value = count_system_state(matrix_d, matrix_f, n, solution)
  return solution, delta, final_value, r, s

In [ ]:
test_cases = {}

In [ ]:
from time import time

In [ ]:
for test_num in range(len(data)):
  print(f'test {test_num}')
  init_solution = None
  best_init_value = 10 ** 9
  matrix_d, matrix_f, n = data[test_num]['D'], data[test_num]['F'], data[test_num]['n']

  for i in range(1000):
    solution = random_init(data[test_num]['n'])
    init_value = count_system_state(matrix_d, matrix_f, n, solution)
    if init_value < best_init_value:
      best_init_value = init_value
      init_solution = solution

  best_final_value = best_init_value
  best_solution = init_solution
  avg_time = 0
  last_delta = None
  last_r, last_s = None, None
  iterations = 1000
  for i in range(iterations):
    start_time = time()
    solution, last_delta, final_value, last_r, last_s = local_search(matrix_d, matrix_f, n, best_solution, last_delta, last_r, last_s)
    end_time = time()
    avg_time += (end_time - start_time)
    if final_value < best_final_value:
      best_final_value = final_value
      best_solution = solution

  avg_time /= iterations
  test_cases[test_num] = {'init_value': best_init_value, 'best_final_value': best_final_value, 'best_solution': best_solution, 'avg_time': avg_time}

test 0
test 1
test 2
test 3
test 4


In [ ]:
test_cases

{0: {'avg_time': 0.0070607497692108155,
  'best_final_value': 812166,
  'best_solution': [12,
   9,
   11,
   1,
   15,
   19,
   4,
   18,
   5,
   2,
   14,
   8,
   3,
   7,
   13,
   16,
   0,
   10,
   6,
   17],
  'init_value': 826000},
 1: {'avg_time': 0.05165911817550659,
  'best_final_value': 3628742,
  'best_solution': [18,
   3,
   30,
   37,
   27,
   26,
   2,
   14,
   25,
   32,
   11,
   23,
   34,
   4,
   9,
   20,
   39,
   1,
   12,
   6,
   8,
   21,
   38,
   0,
   29,
   15,
   31,
   28,
   24,
   13,
   10,
   16,
   22,
   5,
   7,
   36,
   19,
   33,
   35,
   17],
  'init_value': 3628742},
 2: {'avg_time': 0.22635609412193297,
  'best_final_value': 8281558,
  'best_solution': [25,
   20,
   58,
   23,
   29,
   2,
   13,
   52,
   17,
   6,
   26,
   54,
   39,
   8,
   28,
   33,
   5,
   44,
   53,
   11,
   9,
   48,
   55,
   7,
   45,
   43,
   47,
   57,
   42,
   18,
   10,
   51,
   32,
   19,
   14,
   24,
   59,
   38,
   35,
   21,
   56,
   41,


# Табличка ван лав:

In [ ]:
import pandas as pd

In [ ]:
results_1 = {}
for k, v in test_cases.items():
  results_1[k] = [v['avg_time'], v['best_final_value'], v['best_solution']]

In [ ]:
pd.DataFrame.from_dict(results_1, orient='index', columns=['Среднее время выполнения','Конечное значение целевой функции', 'Лучшее решение'])

,Среднее время выполнения,Конечное значение целевой функции,Лучшее решение
0,0.003794,824292,"[13, 9, 15, 4, 19, 10, 1, 6, 12, 0, 17, 18, 8,..."
1,0.047272,3594862,"[3, 10, 21, 9, 38, 4, 36, 15, 19, 2, 22, 30, 3..."
2,0.226775,8294000,"[41, 9, 36, 1, 46, 19, 52, 10, 21, 43, 14, 17,..."
3,0.719522,15243640,"[20, 37, 77, 6, 63, 28, 1, 13, 4, 45, 72, 79, ..."
4,1.697206,23736158,"[49, 38, 16, 11, 15, 24, 23, 62, 30, 19, 79, 4..."


# Iterated local search

In [ ]:
def pertrubation(solution, i, j):
    return solution[:i] + solution[i:j+1][::-1] + solution[j+1:]

In [ ]:
from random import randint

In [ ]:
test_cases_2 = {}

In [ ]:
for test_num in range(len(data)):
  print(f'test {test_num}')
  matrix_d, matrix_f, n = data[test_num]['D'], data[test_num]['F'], data[test_num]['n']
  best_init_value = 10 ** 9

  for i in range(1000):
    solution = random_init(n)
    init_value = count_system_state(matrix_d, matrix_f, n, solution)
    if init_value < best_init_value:
      best_init_value = init_value
      init_solution = solution

  start_time = time()
  init_value = count_system_state(matrix_d, matrix_f, n, init_solution)
  solution, last_delta, final_value, last_r, last_s = local_search(matrix_d, matrix_f, n, init_solution)
  iterations = 2000
  k = 50
  p = 100
  for c in range(iterations):
    i = randint(int(k * n/p) - 10, int(k * n/p))
    j = randint(int((p-k) * n/p) + 1, int((p-k) * n/p) + 10)

    next_solution = pertrubation(solution, i, j)
    next_solution, last_delta, next_final_value, last_r, last_s = local_search(matrix_d, matrix_f, n, next_solution)
    if next_final_value < final_value:
      final_value = next_final_value
      solution = next_solution
      k = 50
      continue
    if k != 10:
      k -= 5
  end_time = time() - start_time

  test_cases_2[test_num] = {'init_value': init_value, 'best_final_value': final_value, 'best_solution': solution, 'time': end_time}

test 0
test 1
test 2
test 3
test 4


In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame.from_dict(test_cases_2, orient='index', columns=['Начальное значение','Конечное значение целевой функции', 'Лучшее решение', 'Время выполнения'])

,Начальное значение,Конечное значение целевой функции,Лучшее решение,Время выполнения


In [ ]:
test_cases_2

{0: {'best_final_value': 839772,
  'best_solution': [6,
   2,
   16,
   11,
   9,
   19,
   13,
   15,
   18,
   8,
   5,
   12,
   3,
   7,
   4,
   1,
   10,
   14,
   17,
   0],
  'init_value': 814650,
  'time': 7.708643674850464},
 1: {'best_final_value': 3658478,
  'best_solution': [13,
   38,
   36,
   25,
   35,
   4,
   18,
   34,
   8,
   39,
   12,
   7,
   10,
   6,
   29,
   11,
   37,
   33,
   27,
   26,
   19,
   23,
   32,
   20,
   17,
   5,
   1,
   28,
   30,
   2,
   24,
   0,
   16,
   21,
   31,
   15,
   9,
   3,
   14,
   22],
  'init_value': 3578244,
  'time': 97.53975892066956},
 2: {'best_final_value': 8312536,
  'best_solution': [57,
   0,
   20,
   18,
   30,
   38,
   1,
   50,
   8,
   41,
   48,
   29,
   5,
   24,
   2,
   6,
   31,
   22,
   7,
   12,
   42,
   59,
   11,
   21,
   35,
   52,
   53,
   9,
   10,
   15,
   51,
   43,
   34,
   55,
   26,
   14,
   45,
   16,
   28,
   47,
   39,
   36,
   17,
   4,
   46,
   32,
   23,
   56,
   13,
   